In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import pickle
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
df = pd.read_excel('/content/drive/My Drive/FLIPR_6/modified_test_dataset.xlsx')
df.head()

,Unnamed: 0,City,State,Type,Population [2011],Popuation [2001],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors
0,0,Tuensang,Nagaland,T.C,36774.0,NaN,931.0,23.0,10.0,MEDIUM,94.0,114.0,0.253390,34237.0,17.0,3131
1,1,Lakshmeshwar,Karnataka,T.M.C,36754.0,NaN,934.0,25.0,38.0,HIGH,62.0,160.0,0.192555,34328.0,13.0,719711
2,2,Zira,Punjab,M.Cl.,36732.0,NaN,883.0,29.0,35.0,HIGH,63.0,105.0,0.887882,32434.0,17.0,274051
3,3,Yawal,Maharashtra,M.Cl,36706.0,NaN,887.0,26.0,31.0,HIGH,60.0,174.0,0.407838,32558.0,11.0,4985284
4,4,Thana Bhawan,Uttar Pradesh,N.P.,36669.0,NaN,877.0,28.0,39.0,LOW,92.0,153.0,0.324456,32159.0,23.0,3509847


In [5]:
df['State'] = df['State'].replace(['Maharashtra', 'Gujarat', 'Andaman and Nicobar Islands', 'Goa', 'Dadra and Nagar Haveli', 'West Bengal'], 'West')
df['State'] = df['State'].replace(['Delhi', 'Rajasthan', 'Uttar Pradesh', 'Punjab', 'Haryana', 'Jammu and Kashmir', 'Chandigarh', 'Uttarakhand', 'Himachal Pradesh' ], 'North')
df['State'] = df['State'].replace(['Karnataka', 'Telangana', 'Tamil Nadu', 'Tamil nadu', 'Kerala','Puducherry'], 'South')
df['State'] = df['State'].replace(['Bihar', 'Jharkhand', 'Assam', 'Odisha', 'Tripura', 'Mizoram', 'Manipur','Sikkim', 'Nagaland', 'Meghalaya', 'Arunachal Pradesh', 'Andhra Pradesh', 'Andhra pradesh'], 'East')
df['State'] = df['State'].replace(['Madhya Pradesh', 'Andhra Pradesh', 'Chhattisgarh'], 'Central')

In [6]:
categorical = ['State']

for var in categorical:
    df = pd.concat([df, 
                    pd.get_dummies(df[var], prefix=var)], axis=1)
    del df[var]

In [7]:
df.head()

,Unnamed: 0,City,Type,Population [2011],Popuation [2001],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors,State_Central,State_East,State_North,State_South,State_West
0,0,Tuensang,T.C,36774.0,NaN,931.0,23.0,10.0,MEDIUM,94.0,114.0,0.253390,34237.0,17.0,3131,0,1,0,0,0
1,1,Lakshmeshwar,T.M.C,36754.0,NaN,934.0,25.0,38.0,HIGH,62.0,160.0,0.192555,34328.0,13.0,719711,0,0,0,1,0
2,2,Zira,M.Cl.,36732.0,NaN,883.0,29.0,35.0,HIGH,63.0,105.0,0.887882,32434.0,17.0,274051,0,0,1,0,0
3,3,Yawal,M.Cl,36706.0,NaN,887.0,26.0,31.0,HIGH,60.0,174.0,0.407838,32558.0,11.0,4985284,0,0,0,0,1
4,4,Thana Bhawan,N.P.,36669.0,NaN,877.0,28.0,39.0,LOW,92.0,153.0,0.324456,32159.0,23.0,3509847,0,0,1,0,0


In [8]:
df.drop(['City', 'Popuation [2001]', 'Type', 'Unnamed: 0'], axis = 'columns', inplace = True)

In [9]:
var = ['Population [2011]','Sex Ratio','Median Age','Avg Temp', 'Toilets Avl', 'Water Purity', 'H Index', 'Female Population', '# of hospitals', 'Foreign Visitors']

for i in range(len(var)):
  df.replace(np.NAN, {var[i]: df[var[i]].median()}, inplace = True)
  

In [10]:
df.isna().sum()

Population [2011]    0
Sex Ratio            0
Median Age           0
Avg Temp             0
SWM                  9
Toilets Avl          0
Water Purity         0
H Index              0
Female Population    0
# of hospitals       0
Foreign Visitors     0
State_Central        0
State_East           0
State_North          0
State_South          0
State_West           0
dtype: int64

In [11]:
df['SWM'].fillna('LOW', inplace = True)

In [12]:

categorical = ['SWM']

for var in categorical:
    df = pd.concat([df, 
                    pd.get_dummies(df[var], prefix=var)], axis=1)
    del df[var]

In [13]:
from sklearn.preprocessing import StandardScaler
continuous = ['Population [2011]','Sex Ratio','Median Age','Avg Temp', 'Toilets Avl', 'Water Purity', 'H Index', 'Female Population', '# of hospitals', 'Foreign Visitors']

scaler = StandardScaler()

for var in continuous:
    df[var] = df[var].astype('float64')
    df[var] = scaler.fit_transform(df[var].values.reshape(-1, 1))

In [14]:
df.isna().sum()

Population [2011]    0
Sex Ratio            0
Median Age           0
Avg Temp             0
Toilets Avl          0
Water Purity         0
H Index              0
Female Population    0
# of hospitals       0
Foreign Visitors     0
State_Central        0
State_East           0
State_North          0
State_South          0
State_West           0
SWM_HIGH             0
SWM_LOW              0
SWM_MEDIUM           0
dtype: int64

In [15]:
df.shape

(501, 18)

In [16]:
file = open("/content/drive/My Drive/FLIPR_6/Task1_model.pickle",'rb')
model = pickle.load(file)

[12:43:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [19]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df.columns.values]

In [22]:
df = df[['# of hospitals', 'Avg Temp', 'Female Population', 'Foreign Visitors', 'H Index', 'Median Age', 'Population _2011_', 'Sex Ratio', 'Toilets Avl', 'Water Purity', 'State_Central', 'State_East', 'State_North', 'State_South', 'State_West', 'SWM_HIGH', 'SWM_LOW', 'SWM_MEDIUM']]

In [23]:
y_pred = model.predict(df)

In [25]:
df1 = pd.read_excel('/content/drive/My Drive/FLIPR_6/modified_test_dataset.xlsx')

In [26]:
result = pd.concat([df1['City'], pd.DataFrame(y_pred)], axis=1)

In [27]:
result.rename(columns={0:"Covid Cases"}, inplace = True)

In [28]:
result.head()

,City,Covid Cases
0,Tuensang,17854.888672
1,Lakshmeshwar,15528.912109
2,Zira,12620.907227
3,Yawal,18204.126953
4,Thana Bhawan,20461.626953


In [29]:
result.to_csv(r'/content/drive/My Drive/FLIPR_6/task2_result.csv', index = False)